<a href="https://colab.research.google.com/github/YianKim/2022_PythonStudy_face_emotion_recognition/blob/main/uncertainty_%EA%B8%B0%EB%B0%98_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글 드라이브


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 라이브러리 불러오기


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import PIL
import pickle
import random
from tqdm import tqdm
from collections import Counter

from keras.layers.core import Lambda
from keras import backend as K

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import BatchNormalization
from keras.regularizers import l2
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import LSTM
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import *
from keras.models import load_model

from tqdm import tqdm

## 데이터 불러오기


In [3]:
fashion_mnist = keras.datasets.fashion_mnist
(data, labels), (test_data, test_labels) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [4]:
train_data = data[range(20000)].reshape([20000,28,28,1])
train_labels = labels[range(20000)].reshape([20000,1])

In [5]:
unlab_data = data[range(20000,60000)].reshape([40000,28,28,1])
unlab_labels = labels[range(20000,60000)].reshape([40000,1])

In [6]:
test_data = test_data.reshape([10000,28,28,1])
test_labels = test_labels.reshape([10000,1])

In [7]:
train_labels2 = []
unlab_labels2 = []
test_labels2 = []

In [8]:
for i in range(20000):
  white = [0,0,0,0,0,0,0,0,0,0]
  white[train_labels[i][0]] = 1
  train_labels2.append(white)

for i in range(40000):
  white = [0,0,0,0,0,0,0,0,0,0]
  white[unlab_labels[i][0]] = 1
  unlab_labels2.append(white)

for i in range(10000):
  white = [0,0,0,0,0,0,0,0,0,0]
  white[test_labels[i][0]] = 1
  test_labels2.append(white)

In [9]:
train_labels = np.array(train_labels2)
unlab_labels = np.array(unlab_labels2)
test_labels = np.array(test_labels2)

## CNN

In [10]:
def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

In [11]:
#######HYPERPARAMATERS###########
epochs = 20
batch_size = 64
learning_rate = 0.001
#################################
  
model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))
model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)

model.fit(
          train_data,
          train_labels,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True
          )

Epoch 1/20
219/219 [==============================] - 11s 8ms/step - loss: 1.7825 - accuracy: 0.6824 - val_loss: 0.6651 - val_accuracy: 0.7638
Epoch 2/20
219/219 [==============================] - 1s 6ms/step - loss: 0.5724 - accuracy: 0.7984 - val_loss: 0.5872 - val_accuracy: 0.7925
Epoch 3/20
219/219 [==============================] - 1s 6ms/step - loss: 0.4923 - accuracy: 0.8217 - val_loss: 0.5521 - val_accuracy: 0.8045
Epoch 4/20
219/219 [==============================] - 1s 6ms/step - loss: 0.4326 - accuracy: 0.8433 - val_loss: 0.5300 - val_accuracy: 0.8162
Epoch 5/20
219/219 [==============================] - 1s 6ms/step - loss: 0.3976 - accuracy: 0.8548 - val_loss: 0.5158 - val_accuracy: 0.8227
Epoch 6/20
219/219 [==============================] - 1s 6ms/step - loss: 0.3829 - accuracy: 0.8610 - val_loss: 0.4926 - val_accuracy: 0.8287
Epoch 7/20
219/219 [==============================] - 1s 6ms/step - loss: 0.3481 - accuracy: 0.8686 - val_loss: 0.4917 - val_accuracy: 0.8308
Epoch

In [12]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')

In [13]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')

## 모델 결과

In [14]:
def model_eval():
  pred_mu = model.predict(test_data)
  for i in range(1, 30):
    pred_mu += model.predict(test_data)
  pred_mu = pred_mu/30
  predicted_test_labels = np.argmax(pred_mu, axis=1)
  return(accuracy_score(np.argmax(test_labels, axis=1), predicted_test_labels))

In [15]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')
model_eval()

0.8918

# 라벨링

## 라벨링; 1

In [16]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')

In [17]:
# Vars : 분산들
# Outs : 결과 값들

Vars = []
Outs = []

temp1 = []

for i in tqdm(range(30)):
  temp1.append(model.predict(unlab_data))

for j in tqdm(range(unlab_data.shape[0])):

  temp2 = np.array([0,0,0,0,0,0,0,0,0,0], 'float32')
  
  for i in range(30):
    temp2 += temp1[i][j]
  Outs.append(temp2/30)
  
  outputs=[]

  for i in range(30):
    outputs.append(temp1[i][j][np.argmax(temp2)])
  Vars.append(np.var(outputs))

100%|██████████| 40000/40000 [00:06<00:00, 6369.08it/s]


In [18]:
Outs = np.array(Outs)

class마다 균등하게 선택

In [19]:
Counter(np.argmax(unlab_labels, axis=1))

Counter({0: 4065,
         1: 3975,
         2: 4018,
         3: 3989,
         4: 4033,
         5: 3990,
         6: 3932,
         7: 3997,
         8: 4029,
         9: 3972})

In [20]:
# upper bound of variance?

UB = []

for h in range(10):
  classvars = []
  for i in tqdm(range(40000)):
    if np.argmax(unlab_labels, axis=1).tolist()[i]==h:
      classvars.append(Vars[i])
  UB.append(np.percentile(classvars, 25))

100%|██████████| 40000/40000 [00:49<00:00, 813.33it/s]


In [21]:
lowvars = []
ind = 0 

for i in Vars:
  if i < UB[np.argmax(unlab_labels, axis=1)[ind]]:
    lowvars.append(ind)
  ind += 1

In [22]:
highvars = []
for i in range(unlab_data.shape[0]):
  if i not in lowvars:
    highvars.append(i)

In [23]:
# 같은 길이의 랜덤하게 고른 data들에 모델로 label 부여 후 정확도 측정
randomindx = random.sample(range(40000), len(lowvars))
accuracy_score(np.argmax(np.array(Outs)[randomindx], axis=1), np.argmax(unlab_labels[randomindx], axis=1))

0.8969

In [24]:
# 저분산의 data들만 모아서 모델로 label 부여 후 정확도 측정 : 95%
accuracy_score(np.argmax(np.array(Outs)[lowvars], axis=1), np.argmax(unlab_labels[lowvars], axis=1))

0.965

In [25]:
train_data_1 = np.concatenate([train_data, unlab_data[lowvars]], axis=0)
train_labels_1 = np.concatenate([train_labels, Outs[lowvars]], axis=0)

In [26]:
shufindx = random.sample(range(train_data_1.shape[0]),train_data_1.shape[0])
train_data_1 = train_data_1[shufindx]
train_labels_1 = train_labels_1[shufindx]

In [27]:
train_data_2 = np.concatenate([train_data, unlab_data[randomindx]], axis=0)
train_labels_2 = np.concatenate([train_labels, Outs[randomindx]], axis=0)

In [28]:
shufindx = random.sample(range(train_data_2.shape[0]),train_data_2.shape[0])
train_data_2 = train_data_2[shufindx]
train_labels_2 = train_labels_2[shufindx]

In [29]:
# unlab_data = unlab_data[highvars]
# unlab_labels = unlab_labels[highvars]

In [30]:
Counter(np.argmax(train_labels_1, axis=1))

Counter({0: 3097,
         1: 3020,
         2: 3077,
         3: 3043,
         4: 3012,
         5: 3008,
         6: 2736,
         7: 3001,
         8: 2984,
         9: 3022})

In [31]:
Counter(np.argmax(train_labels_2, axis=1))

Counter({0: 3023,
         1: 3020,
         2: 3005,
         3: 3064,
         4: 3004,
         5: 3004,
         6: 2873,
         7: 3001,
         8: 2973,
         9: 3033})

## 모델링

간단한 모델에서 받은 지식를 복잡한 모델에서 학습 

uncertainty aware data vs random sampling data

In [32]:
def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

In [33]:
#######HYPERPARAMATERS###########
epochs = 100
batch_size = 64
learning_rate = 0.0001
#################################
  
model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Conv2D(64,(3,3), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)

model.fit(
          train_data_1,
          train_labels_1,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True
          )

Epoch 1/100
329/329 [==============================] - 4s 9ms/step - loss: 4.2144 - accuracy: 0.1091 - val_loss: 2.3682 - val_accuracy: 0.1104
Epoch 2/100
329/329 [==============================] - 2s 7ms/step - loss: 2.3085 - accuracy: 0.1286 - val_loss: 2.2634 - val_accuracy: 0.1397
Epoch 3/100
329/329 [==============================] - 2s 7ms/step - loss: 2.1561 - accuracy: 0.1891 - val_loss: 1.9780 - val_accuracy: 0.2513
Epoch 4/100
329/329 [==============================] - 2s 7ms/step - loss: 1.8091 - accuracy: 0.3191 - val_loss: 1.6900 - val_accuracy: 0.3812
Epoch 5/100
329/329 [==============================] - 2s 7ms/step - loss: 1.6016 - accuracy: 0.4253 - val_loss: 1.5064 - val_accuracy: 0.4703
Epoch 6/100
329/329 [==============================] - 3s 8ms/step - loss: 1.4206 - accuracy: 0.5050 - val_loss: 1.3095 - val_accuracy: 0.5482
Epoch 7/100
329/329 [==============================] - 3s 8ms/step - loss: 1.2121 - accuracy: 0.5873 - val_loss: 1.1052 - val_accuracy: 0.6139

In [34]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn2.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn2.h5')
model_eval()

In [ ]:
#######HYPERPARAMATERS###########
epochs = 100
batch_size = 64
learning_rate = 0.0001
#################################
  
model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Conv2D(64,(3,3), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)

model.fit(
          train_data_2,
          train_labels_2,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True
          )

In [ ]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn_no.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn_no.h5')
model_eval()